# HW 4. Error correction

In [10]:
import pysam
import seaborn as sns
import numpy as np
import pandas as pd

from collections import Counter

## Part 0. Original reads

In [ ]:
!bwa index "./data/error_correction/MG1655-K12.first400K.fasta"

In [ ]:
!fastqc -o ./data/error_correction/fastqc/ "./data/error_correction/ecoli_400K_err_1.fastq.gz" "./data/error_correction/ecoli_400K_err_2.fastq.gz"

<img src="images/per_base_quality_err.png">

In [ ]:
!bwa mem "./data/error_correction/MG1655-K12.first400K.fasta" "./data/error_correction/ecoli_400K_err_1.fastq.gz" "./data/error_correction/ecoli_400K_err_2.fastq.gz" 2>"err.log" | samtools view -bS -o "./data/error_correction/ecoli_400K_err_aligned.bam"

In [ ]:
!samtools index "./data/error_correction/ecoli_400K_err_aligned.bam"

In [15]:
err_alignment = "./data/error_correction/ecoli_400K_err_aligned.bam"
reference_path = "./data/error_correction/MG1655-K12.first400K.fasta"

err_bam = pysam.AlignmentFile(err_alignment, "rb")
contig = err_bam.references[0]
ref_fasta = pysam.FastaFile(reference_path)
seq = ref_fasta.fetch(contig)
print(contig)

gi|49175990|ref|NC_000913.2|


### Алгоритм подстчета ошибок
1) Берем пару изначальный рид, исправленный рид.  
2) Если они не смаппились на один и тот же участок - пропускаем их.  
3) Если смаппились, смотрим по одинаковым позициям на референсе:
* Если основание обрезали, и оно не была верным матчем - Detected and removed error (true positive)  
* Если основание обрезали, и оно была верным матчем - Incorrectly removed base (false positive)
* Если основание не изменилось и было верным матчем - Correctly unmodified base (true negative)  
* Если основание не изменилось и не было верным матчем - Undetected error (false negative)  
* Если основание изменилось и было верным матчем - Falsely corrected error (false positive)  
* Если основание изменилось и стало верным матчем, хотя раньше не было - Detected & corrected error (true positive)   
* Отдельно в этих ридах смотрим на количество инсерций:   
  если стало больше увеличиваем Undetected error (false negative) и Falsely corrected error (false positive)   
  если меньше - Undetected error (false negative) и Detected and removed error (true positive)


In [11]:
def error_correction_stats(corrected_bam, error_bam, contig, seq):
    indexed_corr_reads = pysam.IndexedReads(corrected_bam)
    indexed_corr_reads.build()

    d = {
        'corrected_error': 0, 'removed_error': 0, 
        'undetected_error': 0, 'false_correction': 0, 
        'correctly_unmodified':0, 'incorrectly_removed': 0
    }
    unmapped = 0
    diff_mapping = 0
    total = 0
    unpaired = 0
    for read in err_bam.fetch(contig):
        total += 1
        err_matches = read.get_aligned_pairs(matches_only=False)
        err_seq = read.query_sequence
        try:
            find_reads = indexed_corr_reads.find(read.query_name)
        except KeyError:
            # read was dropped during correction
            unpaired += 1
            continue
            
        cor_read = next(filter(lambda r: r.is_read1 == read.is_read1, find_reads))
        if read.is_unmapped or cor_read.is_unmapped:
            unmapped += 1
            continue
        if read.reference_end < cor_read.reference_end or cor_read.reference_start < read.reference_start:
            diff_mapping += 1
            continue

        cor_seq = cor_read.query_sequence
        if err_seq == cor_seq:
            for read_pos, ref_pos in err_matches:
                if read_pos is None or ref_pos is None or err_seq[read_pos] != seq[ref_pos]:
                    d['undetected_error'] += 1
                else:
                    d['correctly_unmodified'] += 1
            continue

        cor_matches = cor_read.get_aligned_pairs(matches_only=False)

        # count insertions
        insertions_err = sum(int(ref_pos is None) for err_pos, ref_pos in err_matches)
        insertions_cor = sum(int(ref_pos is None) for cor_pos, ref_pos in cor_matches)
        if insertions_err < insertions_cor:
            d['undetected_error'] += insertions_err
            d['false_correction'] += insertions_cor - insertions_err
        else:
            d['undetected_error'] += insertions_cor
            d['removed_error'] += insertions_err - insertions_cor

        ref2err = {ref_pos: err_pos for err_pos, ref_pos in err_matches if ref_pos is not None}
        ref2cor = {ref_pos: cor_pos for cor_pos, ref_pos in cor_matches if ref_pos is not None}
        for pos in ref2err:
            c_ref = seq[pos]
            err_pos = ref2err[pos]
            cor_pos = ref2cor.get(pos, '$')  # '$' marks that position was trimmed
            
            # if position trimmed
            if cor_pos == '$' and err_pos is None:
                d['removed_error'] += 1 
            elif cor_pos == '$' and err_seq[err_pos] != c_ref:
                d['removed_error'] += 1
            elif cor_pos == '$' and err_seq[err_pos] == c_ref:
                d['incorrectly_removed'] += 1

            # if deletion
            elif cor_pos is None and err_pos is None:
                d['undetected_error'] += 1
            elif cor_pos is None and err_seq[err_pos] != c_ref:
                d['undetected_error'] += 1
            elif cor_pos is None and err_seq[err_pos] == c_ref:
                d['false_correction'] += 1

            # if incorrect match
            elif cor_seq[cor_pos] != c_ref and err_pos is None:
                d['undetected_error'] += 1
            elif cor_seq[cor_pos] != c_ref and err_seq[err_pos] != c_ref:
                d['undetected_error'] += 1
            elif cor_seq[cor_pos] != c_ref and err_seq[err_pos] == c_ref:
                d['false_correction'] += 1

            # if correct match
            elif cor_seq[cor_pos] == c_ref and err_pos is None:
                d['corrected_error'] += 1
            elif cor_seq[cor_pos] == c_ref and err_seq[err_pos] != c_ref:
                d['corrected_error'] += 1
            elif cor_seq[cor_pos] == c_ref and err_seq[err_pos] == c_ref:
                d['correctly_unmodified'] += 1
            else:
                print('I did not know there could be more cases ;(')
            
    read_stats = {'n_reads': total, 'unmapped': unmapped, 'position_changed': diff_mapping, 'pair_dropped': unpaired}
    return d, read_stats

## Part 1. Trimmomatic

In [ ]:
!Trimmomatic PE -phred33 ./data/error_correction/ecoli_400K_err_1.fastq.gz ./data/error_correction/ecoli_400K_err_2.fastq.gz -baseout ./data/error_correction/ecoli_400K_trimmed LEADING:10 TRAILING:10 SLIDINGWINDOW:10:10 MINLEN:20

In [ ]:
!fastqc -o ./data/error_correction/fastqc/ "./data/error_correction/ecoli_400K_trimmed_1P" "./data/error_correction/ecoli_400K_trimmed_2P"

<img src="images/per_base_quality_trim.png">

In [ ]:
!bwa mem "./data/error_correction/MG1655-K12.first400K.fasta" "./data/error_correction/ecoli_400K_trimmed_1P" "./data/error_correction/ecoli_400K_trimmed_2P" 2>"trim.log" | samtools view -bS - | samtools sort -o "./data/error_correction/ecoli_400K_trim_aligned.bam"

In [ ]:
!samtools index "./data/error_correction/ecoli_400K_trim_aligned.bam"

In [5]:
trim_alignment = "./data/error_correction/ecoli_400K_trim_aligned.bam"
trim_bam = pysam.AlignmentFile(trim_alignment, "rb")

In [ ]:
def calc_stats(error_bam, corrected_bam, reference_fasta, contig):
    cor_pileup = corrected_bam.pileup(contig, fastafile=reference_fasta)
    err_pileup = error_bam.pileup(contig, fastafile=reference_fasta)

    cor_count = Counter()
    err_count = Counter()

    for cor_col, err_col in zip(cor_pileup, err_pileup):
        cor_match = cor_col.get_query_sequences(mark_matches=True, add_indels=True)
        err_match = err_col.get_query_sequences(mark_matches=True, add_indels=True)
    
        cor_count.update(cor_match)
        err_count.update(err_match)
    
#     print('Corrected reads: ', cor_count)
    print('Orignal reads: ', sum(err_count.values()))
    
    d = {}
    y = err_count - cor_count
    x = cor_count - err_count
    total_removed = sum(err_count.values()) - sum(cor_count.values())
    d['corrected_error'] = x[','] + x['.']
    d['removed_error'] = sum(y[key] for key in y if key not in ['.', ','])
    d['undetected_error'] = sum(min(cor_count[key], err_count[key]) for key in err_count if key not in ['.', ','])
    d['false_correction'] = sum(x[key] for key in x if key not in [',', '.'])
    d['correctly_unmodified'] = min(cor_count['.'], err_count['.']) + min(cor_count[','], err_count[','])
    d['incorrectly_removed'] = total_removed - d['removed_error']
    
    return d

In [ ]:
res = calc_stats(err_bam, trim_bam, ref_fasta, contig)
print(res)

In [17]:
d, read_stats = error_correction_stats(trim_bam, err_bam, contig, seq)
print(d)
print(read_stats)

{'corrected_error': 508, 'removed_error': 6538995, 'undetected_error': 1226955, 'false_correction': 22237, 'correctly_unmodified': 246256706, 'incorrectly_removed': 14665349}
{'n_reads': 2763102, 'unmapped': 882, 'position_changed': 38622, 'pair_dropped': 36736}


| | Error in corrected reads | Correct base in corrected reads| Base is absent in corrected reads |
|--|--|--|--|
|Error in raw data       | 1226955 | 508       | 6538995  |
|Correct base in raw data| 22237   | 246256706 | 14665349 |

In [18]:
print(f'Error % after correction: {100 * (1226955 + 22237) /  (1226955 + 22237 + 246256706 + 508):.2f} %') 
print(f'Error % before correction: {100 * (1226955 + 508 + 6538995) / (1226955 + 508 + 6538995 + 22237 + 246256706 + 14665349):.2f} %')

Error % after correction: 0.50 %
Error % before correction: 2.89 %


## Part 2. Trimmomatic + BayesHammer (SPAdes)

In [4]:
!spades.py -1 ./data/error_correction/ecoli_400K_trimmed_1P.fq -2 ./data/error_correction/ecoli_400K_trimmed_2P.fq --only-error-correction -o ./data/error_correction/spades 2> spades_bh.log

Command line: /Users/b2w/Prog/tools/SPAdes-3.13.0-Darwin/bin/spades.py	-1	/Users/b2w/PycharmProjects/BioinfInstitute/NGS2020/data/error_correction/ecoli_400K_trimmed_1P.fq	-2	/Users/b2w/PycharmProjects/BioinfInstitute/NGS2020/data/error_correction/ecoli_400K_trimmed_2P.fq	--only-error-correction	-o	/Users/b2w/PycharmProjects/BioinfInstitute/NGS2020/data/error_correction/spades	

System information:
  SPAdes version: 3.13.0
  Python version: 3.7.4
  OS: Darwin-19.3.0-x86_64-i386-64bit

Output dir: /Users/b2w/PycharmProjects/BioinfInstitute/NGS2020/data/error_correction/spades
Mode: ONLY read error correction (without assembling)
Debug mode is turned OFF

Dataset parameters:
  Multi-cell mode (you should set '--sc' flag if input data was obtained with MDA (single-cell) technology or --meta flag if processing metagenomic dataset)
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      left reads: ['/Users/b2w/PycharmProjects/BioinfInstitute/NGS2020/data/error_

  0:03:29.448   428M / 3G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 0 produced 44355 new k-mers.
  0:03:55.566   428M / 3G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 1 produced 1633 new k-mers.
  0:04:22.285   428M / 3G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 2 produced 11 new k-mers.
  0:04:46.746   428M / 3G    INFO    General                 (main.cpp                  : 218)   Solid k-mers iteration 3 produced 0 new k-mers.
  0:04:46.746   428M / 3G    INFO    General                 (main.cpp                  : 222)   Solid k-mers finalized
  0:04:46.747   428M / 3G    INFO    General                 (hammer_tools.cpp          : 220)   Starting read correction in 4 threads.
  0:04:46.747   428M / 3G    INFO    General                 (hammer_tools.cpp          : 233)   Correcting pair of reads: /Users/b2w/PycharmProjects/Bioinf

In [5]:
!fastqc -o ./data/error_correction/fastqc/ "./data/error_correction/spades/corrected/ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz" "./data/error_correction/spades/corrected/ecoli_400K_trimmed_2P.00.0_0.cor.fastq.gz"

Started analysis of ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 5% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 10% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 15% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 20% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 25% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 30% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 35% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 40% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 45% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 50% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 55% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 60% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 65% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 70% complete for ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz
Approx 75% comp

In [6]:
!bwa mem "./data/error_correction/MG1655-K12.first400K.fasta" "./data/error_correction/spades/corrected/ecoli_400K_trimmed_1P.00.0_0.cor.fastq.gz" "./data/error_correction/spades/corrected/ecoli_400K_trimmed_2P.00.0_0.cor.fastq.gz" 2>"bh.log" | samtools view -bS - | samtools sort -o "./data/error_correction/ecoli_400K_bayes_aligned.bam"

In [12]:
!samtools index "./data/error_correction/ecoli_400K_bayes_aligned.bam"

In [13]:
bh_alignment = "./data/error_correction/ecoli_400K_bayes_aligned.bam"
bh_bam = pysam.AlignmentFile(bh_alignment, "rb")

In [16]:
bh_correction_stats, bh_read_stats = error_correction_stats(bh_bam, err_bam, contig, seq)
print(bh_correction_stats)
print(bh_read_stats)

{'corrected_error': 816402, 'removed_error': 6307000, 'undetected_error': 320252, 'false_correction': 10348, 'correctly_unmodified': 245089820, 'incorrectly_removed': 14399767}
{'n_reads': 2763102, 'unmapped': 90, 'position_changed': 46036, 'pair_dropped': 47662}


| | Error in corrected reads | Correct base in corrected reads| Base is absent in corrected reads |
|--|--|--|--|
|Error in raw data       | 320252   | 816402      | 6307000  |
|Correct base in raw data| 10348  | 245089820 | 14399767 |

In [19]:
print(f'Error % after correction: {100 * (320252 + 10348) /  (320252 + 816402 + 10348 + 245089820):.2f} %') 
print(f'Error % before correction: {100 * (320252 + 816402 + 6307000) / (320252 + 816402 + 6307000 + 10348 + 245089820 + 14399767):.2f} %')

Error % after correction: 0.13 %
Error % before correction: 2.79 %
